# SQS - Simple Queue Service Notes

## Create a queue

In [ ]:
import boto3

client = boto3.client('sqs')
create_response = client.create_queue(
        QueueName = 'sampleQ'
    )
print(create_response)

## Delete a Queue

In [ ]:
response = client.delete_queue(
    QueueUrl = create_response['QueueUrl']
)

print(response)